In [57]:
import json

pattern_id_to_idx = {}
mapping_id_to_idx = {}

def idx_to_mathcal(idx, map_idx=None):
    idx = chr(ord('A') + idx - 1)
    return "$\\mathcal{" + idx + "}" + (f"_{map_idx}" if map_idx else "") + "$"


In [ ]:

def fuzzing_summary_table(data):
    """
    Generate a LaTeX tabular from the DRAM hammering JSON data,
    using booktabs. Horizontal rules are placed between each
    hammering-pattern block.
    
    Columns:
      - Pattern # (1...n)
      - Num activations
      - Num refresh intervals (if available; else N/A)
      - Mapping ID
      - Bank No
      - Min Row
      - Max Row
      - # Bit Flips
    """
    headers = [
        "Pattern",
        "Activations",
        r"\shortstack[c]{Refresh\\Intervals}",
        "Mapping Identifier",
        "Bank No.",
        "Min Row",
        "Max Row",
        "\\# Bit Flips"
    ]
    
    # Begin LaTeX table
    col_format = "".join(["c"] * len(headers))
    latex = [
        f"\\begin{{tabular}}{{{col_format}}}",
        "\\toprule",
        " & ".join(headers) + " \\\\",
        "\\midrule"
    ]
    
    # Iterate each hammering pattern
    for pat_num, pat in enumerate(data.get("hammering_patterns", []), start=1):
        activations = len(pat.get("access_ids", []))
        refresh = pat.get("num_refresh_intervals", "N/A")
        
        # convert pattern index to mathcal and record lookup
        math_idx = idx_to_mathcal(pat_num)
        pattern_id_to_idx[pat["id"]] = pat_num
        
        # For each address mapping under this pattern
        for map_idx, mapping in enumerate(pat.get("address_mappings", []), start=1):
            min_row = mapping.get("min_row", "N/A")
            max_row = mapping.get("max_row", "N/A")
            flips = sum(len(phase) for phase in mapping.get("bit_flips", []))
            bank = mapping.get("bank_no", "N/A")
            
            # record mapping lookup
            mapping_id_to_idx[mapping["id"]] = map_idx
            
            # only print the pattern-level cols on the first mapping row
            row_idx = math_idx
            row_acts = str(activations)
            row_ref = str(refresh)
            math_idx = ""      # clear for subsequent rows
            activations = ""   # clear for subsequent rows
            refresh = ""       # clear for subsequent rows
            
            row = [
                row_idx,
                row_acts,
                row_ref,
                str(map_idx),
                str(bank),
                str(min_row),
                str(max_row),
                str(flips)
            ]
            latex.append("  " + " & ".join(row) + " \\\\")
        
        # after each pattern block, add a midrule
        latex.append("  \\midrule")
    
    # replace the last midrule with bottomrule
    if latex[-1].strip() == "\\midrule":
        latex[-1] = "\\bottomrule"
    else:
        latex.append("\\bottomrule")
    
    latex.append("\\end{tabular}")
    return "\n".join(latex)


In [59]:
import json

with open('../config/fuzz-summary.json') as f:
    data = json.load(f)

print(fuzzing_summary_table(data))

\begin{tabular}{cccccccc}
\toprule
Pattern & Activations & \shortstack[c]{Refresh\\Intervals} & Mapping Identifier & Bank No. & Min Row & Max Row & \# Bit Flips \\
\midrule
  $\mathcal{A}$ & 304 & 4 & 1 & 21 & 662 & 821 & 1 \\
  \midrule
  $\mathcal{B}$ & 624 & 8 & 1 & 0 & 1486 & 1635 & 1 \\
   &  &  & 2 & 2 & 400 & 478 & 1 \\
  \midrule
  $\mathcal{C}$ & 624 & 8 & 1 & 18 & 2061 & 2220 & 2 \\
  \midrule
  $\mathcal{D}$ & 624 & 8 & 1 & 11 & 2827 & 2986 & 1 \\
   &  &  & 2 & 13 & 2944 & 3102 & 0 \\
   &  &  & 3 & 17 & 756 & 914 & 0 \\
  \midrule
  $\mathcal{E}$ & 304 & 4 & 1 & 31 & 780 & 938 & 1 \\
   &  &  & 2 & 1 & 1340 & 1489 & 0 \\
  \midrule
  $\mathcal{F}$ & 304 & 4 & 1 & 30 & 455 & 614 & 2 \\
   &  &  & 2 & 2 & 1157 & 1315 & 0 \\
  \midrule
  $\mathcal{G}$ & 312 & 4 & 1 & 27 & 1177 & 1315 & 1 \\
   &  &  & 2 & 29 & 2345 & 2504 & 0 \\
   &  &  & 3 & 31 & 1289 & 1427 & 0 \\
  \midrule
  $\mathcal{H}$ & 312 & 4 & 1 & 23 & 684 & 842 & 1 \\
\bottomrule
\end{tabular}


In [60]:
import json

def summarize_reproducibility(json_data):
    """Extracts summary statistics from a single reproducibility result."""
    bitflips = json_data["reproducibility"]["bitflips_per_round"]
    retries = json_data["reproducibility"]["retries_per_round"]
    times_us = json_data["reproducibility"]["time_per_round_us"]
    total_time_s = json_data["reproducibility"]["total_time_us"] / 1e6
    pattern_id = json_data["reproducibility"]["pattern_id"]
    pattern_id = pattern_id_to_idx[pattern_id]
    mapping_id = json_data["reproducibility"]["mapping_id"]
    mapping_id = mapping_id_to_idx[mapping_id]
    
    return {
        "Candidate": idx_to_mathcal(pattern_id, mapping_id),
        "Avg. Bitflips/Round": round(sum(bitflips) / len(bitflips), 2),
        "Max Bitflips/Round": max(bitflips),
        "Avg. Retries": round(sum(retries) / len(retries), 2),
        "Max Retries": max(retries),
        "Avg. Time (s)": round(sum(times_us) / len(times_us) / 1e6, 2),
        "Max Time (s)": round(max(times_us) / 1e6, 2),
        "Total Time (s)": round(total_time_s, 2)
    }

In [61]:
def reproducibility_table(json_files):
    """Generates a LaTeX booktabs table from a list of JSON filenames with explicit spacing between column groups."""
    SPACING = "\\hspace{2em}"  # Constant for inter-group spacing

    table_rows = []
    for filename in json_files:
        with open(filename, "r") as f:
            data = json.load(f)
        summary = summarize_reproducibility(data)
        table_rows.append(summary)

    table_rows = sorted(table_rows, key=lambda x: x["Candidate"])

    header = [
        "Candidate", "Avg. Bitflips/Round", "Max Bitflips/Round",
        "Avg. Retries", "Max Retries",
        "Avg. Time (s)", "Max Time (s)", "Total Time (s)"
    ]

    # Column format with spacing between groups
    col_format = (
        "l"
        f"@{{{SPACING}}}cc"
        f"@{{{SPACING}}}cc"
        f"@{{{SPACING}}}ccc"
    )

    # Begin LaTeX table
    latex = f"\\begin{{tabular}}{{{col_format}}}\n"
    latex += "\\toprule\n"
    latex += (
        f"Candidate & \\multicolumn{{2}}{{c@{{{SPACING}}}}}{{Bitflips}} & "
        f"\\multicolumn{{2}}{{c@{{{SPACING}}}}}{{Retries}} & "
        "\\multicolumn{3}{c}{Time (s)} \\\\\n"
    )
    # Add cmidrule lines under multicolumn groups
    latex += (
        "\\cmidrule(l){2-3}"
        "\\cmidrule(l){4-5}"
        "\\cmidrule(l){6-8}\n"
    )
    latex += " & Avg. & Max & Avg. & Max & Avg. & Max & Total \\\\\n"
    latex += "\\midrule\n"

    # Rows
    for row in table_rows:
        latex += " & ".join(str(row[h]) for h in header) + " \\\\\n"

    latex += "\\bottomrule\n\\end{tabular}"
    return latex


In [62]:
import glob
print(reproducibility_table(glob.glob("../../blacksmith/build/reproducibility-experiment-*.json")))

\begin{tabular}{l@{\hspace{2em}}cc@{\hspace{2em}}cc@{\hspace{2em}}ccc}
\toprule
Candidate & \multicolumn{2}{c@{\hspace{2em}}}{Bitflips} & \multicolumn{2}{c@{\hspace{2em}}}{Retries} & \multicolumn{3}{c}{Time (s)} \\
\cmidrule(l){2-3}\cmidrule(l){4-5}\cmidrule(l){6-8}
 & Avg. & Max & Avg. & Max & Avg. & Max & Total \\
\midrule
$\mathcal{A}_1$ & 1.3 & 2 & 1.4 & 4 & 1.22 & 3.54 & 12.2 \\
$\mathcal{B}_1$ & 0.0 & 0 & 1000.0 & 1000 & 901.1 & 901.45 & 9011.03 \\
$\mathcal{C}_1$ & 0.0 & 0 & 1000.0 & 1000 & 883.49 & 885.65 & 8834.95 \\
$\mathcal{D}_1$ & 2.1 & 6 & 8.9 & 26 & 7.93 & 23.13 & 79.26 \\
$\mathcal{E}_1$ & 1.1 & 2 & 271.6 & 780 & 236.13 & 678.13 & 2361.32 \\
$\mathcal{F}_1$ & 0.2 & 1 & 863.9 & 1000 & 774.79 & 896.88 & 7747.86 \\
$\mathcal{G}_1$ & 1.0 & 1 & 23.7 & 51 & 20.23 & 43.53 & 202.3 \\
$\mathcal{H}_1$ & 1.1 & 2 & 10.8 & 34 & 9.52 & 29.97 & 95.19 \\
\bottomrule
\end{tabular}


In [63]:
pattern_id_to_idx

{'aaf3a69a-de8f-4151-90d2-50b4d8f41f70': 1,
 'ba30cca4-bf30-44f4-8817-50200d95ac8a': 2,
 'ed34b710-415a-4d73-8a63-0a6c6a4b3d52': 3,
 'df500365-1998-493c-b2ff-887c9c616e6c': 4,
 '1837071a-c0f0-4bcd-938a-1f3970b135d2': 5,
 'fd318369-3672-4047-95e1-6976cad77fb9': 6,
 '39ad622b-3bfe-4161-b860-dad5f3e6dd68': 7,
 '079116ff-d86e-4f56-a005-0c4c64566fce': 8}

In [64]:
mapping_id_to_idx

{'48db9269-682a-40a3-b18f-7db706a5983f': 1,
 '6410b7a8-7587-462d-8f6e-21a5363e595b': 1,
 '212d05bf-91c4-4335-b167-a5ecad0ef5c1': 2,
 'b79f5d95-7710-4873-8baf-80776074d303': 1,
 '064be0f8-2cf4-4c11-92fa-99ffb4e04d9f': 1,
 '53bec1a2-8a29-4821-b7a0-e99376ad48ac': 2,
 '85dadb91-6115-4f22-9757-5cae1817dd07': 3,
 '5fa07388-7ab1-4d5c-a0af-900e9082f944': 1,
 'aa9b0de0-22d1-44a8-a463-31b30f2082e7': 2,
 '7e2b99c3-66c3-4a16-8bc7-7b74dabdfdc6': 1,
 '5b0b66a9-7803-4ee6-9525-fdde204523f3': 2,
 '424c5a1b-6639-4d5d-93f6-287b6de221a6': 1,
 '4d16a3db-c991-419a-b6fe-3a7f41113a8e': 2,
 'c9355362-46cf-4d4a-8e2d-1dcbc4ee00c9': 3,
 '76b95b43-9657-4446-9cbf-329772f5da1d': 1}